In [ ]:
import cv2
import numpy as np

def load_image(path):
    img = cv2.imread(path)
    if img is None:
        raise FileNotFoundError(f"Image not found at path: {path}")
    print(f"Image loaded: {path}, shape: {img.shape}")
    return img

def create_carrot_mask(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_carrot = np.array([12, 150, 150])
    upper_carrot = np.array([20, 255, 255])
    mask = cv2.inRange(hsv, lower_carrot, upper_carrot)
    return mask

def create_rotten_mask(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lower_dark = np.array([0, 0, 0])
    upper_dark = np.array([50, 255, 180]) 
    dark_mask = cv2.inRange(hsv, lower_dark, upper_dark)
    lower_white = np.array([0, 0, 180])
    upper_white = np.array([180, 60, 255])
    white_mask = cv2.inRange(hsv, lower_white, upper_white)
    _, gray_mask = cv2.threshold(gray, 90, 255, cv2.THRESH_BINARY_INV)
    combined = cv2.bitwise_or(dark_mask, white_mask)
    combined = cv2.bitwise_or(combined, gray_mask)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    cleaned = cv2.morphologyEx(combined, cv2.MORPH_OPEN, kernel)
    return cleaned

def is_carrot_present(mask_carrot, img_shape):
    total_pixels = np.sum(mask_carrot > 0)
    ratio = total_pixels / (img_shape[0] * img_shape[1])
    return total_pixels > 800 and 0.005 < ratio < 0.6

def classify_freshness(img):
    carrot_mask = create_carrot_mask(img)
    rotten_mask = create_rotten_mask(img)
    rotten_in_carrot = cv2.bitwise_and(carrot_mask, rotten_mask)
    total_carrot = np.sum(carrot_mask > 0)
    total_rotten = np.sum(rotten_in_carrot > 0)
    if total_carrot == 0 or not is_carrot_present(carrot_mask, img.shape):
        return "Not a carrot or unclear", carrot_mask, rotten_in_carrot
    rotten_ratio = total_rotten / total_carrot
    print(f"Total carrot pixels: {total_carrot}, Rotten pixels inside carrot: {total_rotten}, Rotten ratio: {rotten_ratio:.4f}")
    if rotten_ratio > 0.07 and total_rotten > 150:
        return f"Not Fresh ({(1 - rotten_ratio)*100:.1f}% good)", carrot_mask, rotten_in_carrot
    else:
        return f"Fresh ({(1 - rotten_ratio)*100:.1f}% good)", carrot_mask, rotten_in_carrot

def display_results(img, carrot_mask, rotten_mask, classification):
    marked = img.copy()
    marked[rotten_mask > 0] = [0, 0, 255] 
    cv2.imshow("Original Image", img)
    cv2.imshow("Carrot Mask", carrot_mask)
    cv2.imshow("Rotten Mask", rotten_mask)
    cv2.imshow("Rotten Inside Carrot Mask", rotten_mask)
    cv2.imshow(f"Result - {classification}", marked)
    print(f"Classification: {classification}")
    print("Press any key in an image window to exit...")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    path = input("Enter image path: ").strip()
    image = load_image(path)
    classification, carrot_mask, rotten_mask = classify_freshness(image)
    display_results(image, carrot_mask, rotten_mask, classification)


Image loaded: C:\Users\Destiny\Downloads\Rotten Carrot\download.jpg, shape: (183, 275, 3)
Total carrot pixels: 2227, Rotten pixels inside carrot: 178, Rotten ratio: 0.0799
Classification: Not Fresh (92.0% good)
Press any key in an image window to exit...
